# MoonDAO Governance Proposals and Decisions

This file extracts data about proposals from MoonDAO's [snapshot](https://snapshot.org/#/tomoondao.eth) and saves to csv for future analysis

In [ ]:
#this project uses selenium and pandas libra
#installing 
!pip3 install selenium
!pip3 install pandas

In [ ]:
#downloading and unzipping driver
#get url for your system from this link https://sites.google.com/chromium.org/driver/downloads
#select the right driver from the link above based on your OS and Chrome version
!wget [find url in the link above]
!unzip [insert filename of the downloaded driver]

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time 


In [ ]:
#run instance of Chrome
wd = webdriver.Chrome("./chromedriver")

In [ ]:
#Get all proposals
proposals_url = "https://snapshot.org/#/tomoondao.eth"

wd.get(proposals_url)
time.sleep(2)

b = wd.find_element(by=By.TAG_NAME, value="body")

for i in range(100): #maybe set >100 
    time.sleep(0.1)
    b.send_keys(Keys.PAGE_DOWN)


In [ ]:
#all of the projects are here
props = wd.find_elements_by_css_selector(".bg-skin-block-bg.border-b")


In [ ]:
#var to store extracted info 
proposals = []

#extracting information about projects and storing it in the results
for prop in props:
    author = prop.find_element_by_css_selector("span.truncate").text
    core = prop.find_element_by_css_selector(".border.ml-1").text
    status = prop.find_element_by_css_selector("span.State").text
    title = prop.find_element_by_css_selector(".break-words.my-1").text
    preview = prop.find_element_by_css_selector(".break-words.mb-2").text
    link = prop.find_element_by_css_selector("a.block.p-3").get_attribute("href")
    
    result = prop.find_elements_by_css_selector("span.flex span")
    if result:
        result = result[0].text
    
    proposals.append({
        "author": author,
        "affects": core,
        "status": status,
        "title": title,
        "preview": preview,
        "link": link,
        "result" : result,
    })

In [ ]:
#Open each project's page to extract additional information
for prop in proposals:
    wd.get(prop['link'])
    time.sleep(2)
    
    ipfs = wd.find_elements_by_css_selector("a.float-right.whitespace-nowrap")[0].get_attribute('href')
    snapshot = wd.find_elements_by_css_selector("a.float-right.whitespace-nowrap")[1].get_attribute('href')
    author_nick = wd.find_elements_by_css_selector(".truncate.w-full")[0].text
    
    discussion = wd.find_elements_by_css_selector(".py-4 a.whitespace-nowrap")
    if len(discussion):
        discussion = discussion[0].get_attribute('href')

    text = wd.find_element_by_css_selector("div.break-words").text
    voting_system = wd.find_elements_by_css_selector("span.float-right.text-skin-link")[1].text
    start_date = wd.find_elements_by_css_selector("span.float-right.text-skin-link")[2].text
    end_data = wd.find_elements_by_css_selector("span.float-right.text-skin-link")[3].text
    
    b = wd.find_element_by_tag_name("body")
    for i in range(20):
        time.sleep(0.1)
        b.send_keys(Keys.PAGE_DOWN)
        
    votes = wd.find_element_by_css_selector(".bg-skin-text").text
    
    prop["ipfs"] = ipfs
    prop["snapshot"] = snapshot
    prop["discussion"] = discussion
    prop["text"] = text
    prop["voting_system"] = voting_system
    prop["start_date"] = start_date
    prop["end_data"] = end_data
    prop["votes"] = votes
    prop["author_nick"] = author_nick 
    
    

In [ ]:
#Save results to csv
data = pd.DataFrame(proposals)

data = data[::-1]  #reversing order so rows are chronological

data.to_csv("moondao-data.csv", index=False)